In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import pandas as pd

data = pd.read_csv('/content/drive/My Drive/SentimentAnalysis/rnn/data/clean_data.csv', error_bad_lines=False)
data.CleanText = data.CleanText.astype(str)
data.head()

,ItemID,Sentiment,SentimentSource,SentimentText,CleanText
0,1,0,Sentiment140,is so sad for my APL frie...,is so sad for my apl friend
1,2,0,Sentiment140,I missed the New Moon trail...,i missed the new moon trailer
2,3,1,Sentiment140,omg its already 7:30 :O,omg its already 7 30 o
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...,omgaga im sooo im gunna cry i ve been at this ...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...,i think mi bf is cheating on me t t


In [0]:
import pickle

# load tokenizer
with open('/content/drive/My Drive/SentimentAnalysis/rnn/tokenizers/tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)

In [4]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

x_train, x_test, y_train, y_test = train_test_split(data['CleanText'], 
                                                    data['Sentiment'], 
                                                    test_size=0.1, 
                                                    random_state=42,
                                                    stratify=data['Sentiment'])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

Using TensorFlow backend.


(1420750,) (157862,) (1420750,) (157862,)


In [5]:
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

MAX_LENGTH = 35
padded_train_sequences = pad_sequences(train_sequences, maxlen=MAX_LENGTH)
padded_test_sequences = pad_sequences(test_sequences, maxlen=MAX_LENGTH)

padded_train_sequences.shape

(1420750, 35)

In [6]:
import numpy as np
import sys

embeddings_index = {}
f = open('/content/drive/My Drive/SentimentAnalysis/rnn/embeddings/glove.840B.300d.txt')
f.tell()
i = 0
for line in f:
  values = line.split()
  word = values[0]
  try:
    coefs = np.asarray(values[1:], dtype=np.float32)
  except ValueError as e:
    print("\nValueError: ", e, "\n")
  embeddings_index[word] = coefs
  i += 1
  sys.stdout.write("\r%d%%" % i)
  sys.stdout.flush()
  
f.close()
print('*** Done! In Total, loaded %s word vectors ***' % len(embeddings_index))

128261%
ValueError:  could not convert string to float: 'name@domain.com' 

151102%
ValueError:  could not convert string to float: '.' 

209833%
ValueError:  could not convert string to float: '.' 

220779%
ValueError:  could not convert string to float: '.' 

253461%
ValueError:  could not convert string to float: 'name@domain.com' 

365745%
ValueError:  could not convert string to float: 'name@domain.com' 

532048%
ValueError:  could not convert string to float: 'name@domain.com' 

717302%
ValueError:  could not convert string to float: 'name@domain.com' 

994818%
ValueError:  could not convert string to float: 'name@domain.com' 

1123331%
ValueError:  could not convert string to float: 'Killerseats.com' 

1499727%
ValueError:  could not convert string to float: 'name@domain.com' 

2196017%*** Done! In Total, loaded 2195884 word vectors ***


In [0]:
import pickle

# load tokenizer, used in the simple rnn model previously 
with open('/content/drive/My Drive/SentimentAnalysis/rnn/tokenizers/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [0]:
embed_size = 300
# maximum number of words kept after tokenization based on their word frequency
MAX_NB_WORDS = 80000

nb_words = MAX_NB_WORDS
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((nb_words, embed_size))
for word, index in tokenizer.word_index.items():
  if index > nb_words - 1:
    break
  else:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[index] = embedding_vector

In [14]:
embedding_matrix.shape

(80000, 300)

In [0]:
import numpy as np 
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, GRU, Bidirectional
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate

MAX_NB_WORDS = 80000

def get_rnn_model_with_glove_embeddings():
  embedding_dim = 300
  inp = Input(shape=(MAX_LENGTH, ))
  x = Embedding(MAX_NB_WORDS, embedding_dim, weights=[embedding_matrix], input_length=MAX_LENGTH, trainable=True)(inp)
  x = SpatialDropout1D(0.3)(x)
  x = Bidirectional(GRU(100, return_sequences=True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)

  model = Model(inputs=inp, outputs=outp)
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [26]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

rnn_model_with_embeddings = get_rnn_model_with_glove_embeddings()

filepath="/content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-glove-model-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

batch_size = 256
epochs = 4

history = rnn_model_with_embeddings.fit(x=padded_train_sequences, 
                    y=y_train, 
                    validation_data=(padded_test_sequences, y_test), 
                    batch_size=batch_size, 
                    callbacks=[checkpoint], 
                    epochs=epochs, 
                    verbose=1)

Train on 1420750 samples, validate on 157862 samples
Epoch 1/4
1420750/1420750 [==============================] - 1358s 956us/step - loss: 0.3998 - acc: 0.8173 - val_loss: 0.3691 - val_acc: 0.8362

Epoch 00001: val_acc improved from -inf to 0.83617, saving model to /content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-glove-model-01-0.8362.hdf5
Epoch 2/4
1420750/1420750 [==============================] - 1366s 962us/step - loss: 0.3533 - acc: 0.8435 - val_loss: 0.3608 - val_acc: 0.8406

Epoch 00002: val_acc improved from 0.83617 to 0.84063, saving model to /content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-glove-model-02-0.8406.hdf5
Epoch 3/4
1420750/1420750 [==============================] - 1363s 959us/step - loss: 0.3277 - acc: 0.8570 - val_loss: 0.3655 - val_acc: 0.8396

Epoch 00003: val_acc did not improve from 0.84063
Epoch 4/4
1420750/1420750 [==============================] - 1288s 906us/step - loss: 0.3062 - acc: 0.8684 - val_loss: 0.3729 - val_acc: 0.8384

Epoch 0000

In [10]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

rnn_model_GloVe = load_model('/content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-glove-model-01-0.8362.hdf5')

y_pred = rnn_model_GloVe.predict(padded_test_sequences, verbose=1, batch_size=2048)

y_pred = pd.DataFrame(y_pred, columns=['prediction'])
y_pred['prediction'] = y_pred['prediction'].map(lambda p: 1 if p >= 0.5 else 0)
print('Accuracy of test set: ', accuracy_score(y_test, y_pred)*100, '%')

157862/157862 [==============================] - 8s 48us/step
Accuracy of test set:  83.61733666113442 %
